In [9]:
import requests
from requests_oauthlib import OAuth2Session
from io import StringIO
import nbformat
import chardet
import re

In [10]:
client_id = "bb35165a1f39f8a5f35e"
client_secret = "633e332b09f92d4fc7bc9d82aa90005c0b1e1036"
authorization_base_url = 'https://github.com/login/oauth/authorize'
token_url = 'https://github.com/login/oauth/access_token'
redirect_uri = 'https://localhost:8000/callback'
block_size = 2048

In [11]:
def get_github_access_token(client_id, client_secret, authorization_response):
    github = OAuth2Session(client_id, redirect_uri=redirect_uri)
    token = github.fetch_token(token_url, authorization_response=authorization_response, client_secret=client_secret)
    return token['access_token']

In [12]:
def extract_python_code_from_ipynb(ipynb_content):
    notebook = nbformat.reads(ipynb_content.getvalue(), as_version=4)
    code_cells = [cell['source'] for cell in notebook['cells'] if cell['cell_type'] == 'code']
    return '\n'.join(code_cells)

In [13]:
def get_repo_contents(repo_url, path='', access_token=None):
    _, username, repo_name = repo_url.rstrip('/').rsplit('/', 2)
    user_input = f"I will provide you code for the repository {repo_name} in the next prompts, keep it in chat history"
    main_list = []
    main_list.append(user_input)

    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{path}"

    headers = {}
    if access_token:
        headers['Authorization'] = f"Bearer {access_token}"

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        contents = response.json()
        
        print(contents)

        for i in contents:
            if i:
                if 'type' in i and i['type'] == 'file' and i['name'].lower().endswith(('.py', '.js', '.java', '.c', '.cpp', '.cs', '.html', '.css', '.rb', '.php', '.swift', '.m', '.sql', '.sh', '.ts', '.go', '.rs', '.kt', '.vue', '.jsx', '.ipynb')):
                    file_content_url = i['download_url']
                    file_response = requests.get(file_content_url, headers=headers)

                    if file_response.status_code == 200:
                        content_input = f"This is the code for {i['name']}"
                        main_list.append(content_input)
                        if i['name'].lower().endswith('.ipynb'):
                            ipynb_content = StringIO(file_response.content.decode('utf-8', errors='replace'))
                            python_code = extract_python_code_from_ipynb(ipynb_content)
                            while python_code:
                                block = python_code[:block_size]
                                block_input = block
                                main_list.append(block_input)
                                python_code = python_code[block_size:]
                        else:
                            encoding = chardet.detect(file_response.content)['encoding']
                            encoding = encoding if encoding else 'utf-8'
                            file_content = file_response.content.decode(encoding, errors='replace')
                            while file_content:
                                block = file_content[:block_size]
                                block_input = block
                                main_list.append(block_input)
                                file_content = file_content[block_size:]
                    elif 'type' in i and i['type'] == 'dir':
                        folder_input = f"This is content in folder{i}"
                        main_list.append(folder_input)
                        get_repo_contents(repo_url, path=f"{path}/{i}", access_token=access_token)
                        root_input = "Now we are back in the root directory of the repository"
                        main_list.append(root_input)
        s1 = f"This is all the pieces of code I have for {repo_name}, now I will ask you questions about {repo_name}"
        main_list.append(s1)
    else:
        print(f"Failed to fetch repository contents. Status code: {response.status_code}")

    return main_list

In [14]:
def get_repo_list(access_token):
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get('https://api.github.com/user/repos', headers=headers)
    if response.status_code == 200:
        print(response.json())
        return response.json()
    else:
        print(f"Failed to fetch repository list. Status code: {response.status_code}")
        return None

In [15]:
github_username = input("Enter your GitHub username: ")

In [16]:
github = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=['repo'])
authorization_url, state = github.authorization_url(authorization_base_url)
print(f'Please go to {authorization_url} and authorize access.')

Please go to https://github.com/login/oauth/authorize?response_type=code&client_id=bb35165a1f39f8a5f35e&redirect_uri=https%3A%2F%2Flocalhost%3A8000%2Fcallback&scope=repo&state=9CMFWKbfdqO1hcMpo3NgguvbFZt63F and authorize access.


In [ ]:
authorization_response = input('Paste the full redirect URL here: ')
token = get_github_access_token(client_id, client_secret, authorization_response)

repo_list = get_repo_list(token)

print(repo_list)

In [ ]:
if repo_list:
    print("Repositories:")
    for repo in repo_list:
        print(repo['name'])
    print()

    selected_repo = input("Enter the name of the repository you want to access: ")

    u = get_repo_contents(f"https://github.com/{github_username}/{selected_repo}", access_token=token)
    print(u)

In [1]:
now = [
  {
    'id': 746748586,
    'node_id': 'R_kgDOLIJ6qg',
    'name': 'defy',
    'full_name': 'Akshat723/defy',
    'private': False,
    'owner': {
      'login': 'Akshat723',
      'id': 143522627,
      'node_id': 'U_kgDOCI37Qw',
      'avatar_url': 'https://avatars.githubusercontent.com/u/143522627?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/Akshat723',
      'html_url': 'https://github.com/Akshat723',
      'followers_url': 'https://api.github.com/users/Akshat723/followers',
      'following_url': 'https://api.github.com/users/Akshat723/following{/other_user}',
      'gists_url': 'https://api.github.com/users/Akshat723/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/Akshat723/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/Akshat723/subscriptions',
      'organizations_url': 'https://api.github.com/users/Akshat723/orgs',
      'repos_url': 'https://api.github.com/users/Akshat723/repos',
      'events_url': 'https://api.github.com/users/Akshat723/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/Akshat723/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/Akshat723/defy',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/Akshat723/defy',
    'forks_url': 'https://api.github.com/repos/Akshat723/defy/forks',
    'keys_url': 'https://api.github.com/repos/Akshat723/defy/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/Akshat723/defy/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/Akshat723/defy/teams',
    'hooks_url': 'https://api.github.com/repos/Akshat723/defy/hooks',
    'issue_events_url': 'https://api.github.com/repos/Akshat723/defy/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/Akshat723/defy/events',
    'assignees_url': 'https://api.github.com/repos/Akshat723/defy/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/Akshat723/defy/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/Akshat723/defy/tags',
    'blobs_url': 'https://api.github.com/repos/Akshat723/defy/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/Akshat723/defy/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/Akshat723/defy/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/Akshat723/defy/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/Akshat723/defy/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/Akshat723/defy/languages',
    'stargazers_url': 'https://api.github.com/repos/Akshat723/defy/stargazers',
    'contributors_url': 'https://api.github.com/repos/Akshat723/defy/contributors',
    'subscribers_url': 'https://api.github.com/repos/Akshat723/defy/subscribers',
    'subscription_url': 'https://api.github.com/repos/Akshat723/defy/subscription',
    'commits_url': 'https://api.github.com/repos/Akshat723/defy/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/Akshat723/defy/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/Akshat723/defy/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/Akshat723/defy/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/Akshat723/defy/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/Akshat723/defy/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/Akshat723/defy/merges',
    'archive_url': 'https://api.github.com/repos/Akshat723/defy/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/Akshat723/defy/downloads',
    'issues_url': 'https://api.github.com/repos/Akshat723/defy/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/Akshat723/defy/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/Akshat723/defy/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/Akshat723/defy/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/Akshat723/defy/labels{/name}',
    'releases_url': 'https://api.github.com/repos/Akshat723/defy/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/Akshat723/defy/deployments',
    'created_at': '2024-01-22T15:45:49Z',
    'updated_at': '2024-01-23T07:09:22Z',
    'pushed_at': '2024-01-24T04:46:31Z',
    'git_url': 'git://github.com/Akshat723/defy.git',
    'ssh_url': 'git@github.com:Akshat723/defy.git',
    'clone_url': 'https://github.com/Akshat723/defy.git',
    'svn_url': 'https://github.com/Akshat723/defy',
    'homepage': None,
    'size': 26,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': 'JavaScript',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'gpl-3.0',
      'name': 'GNU General Public License v3.0',
      'spdx_id': 'GPL-3.0',
      'url': 'https://api.github.com/licenses/gpl-3.0',
      'node_id': 'MDc6TGljZW5zZTk='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 787175308,
    'node_id': 'R_kgDOLutXjA',
    'name': 'GiAN',
    'full_name': 'AmmanChhetri/GiAN',
    'private': False,
    'owner': {
      'login': 'AmmanChhetri',
      'id': 121025542,
      'node_id': 'U_kgDOBza0Bg',
      'avatar_url': 'https://avatars.githubusercontent.com/u/121025542?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/AmmanChhetri',
      'html_url': 'https://github.com/AmmanChhetri',
      'followers_url': 'https://api.github.com/users/AmmanChhetri/followers',
      'following_url': 'https://api.github.com/users/AmmanChhetri/following{/other_user}',
      'gists_url': 'https://api.github.com/users/AmmanChhetri/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/AmmanChhetri/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/AmmanChhetri/subscriptions',
      'organizations_url': 'https://api.github.com/users/AmmanChhetri/orgs',
      'repos_url': 'https://api.github.com/users/AmmanChhetri/repos',
      'events_url': 'https://api.github.com/users/AmmanChhetri/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/AmmanChhetri/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/AmmanChhetri/GiAN',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/AmmanChhetri/GiAN',
    'forks_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/forks',
    'keys_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/teams',
    'hooks_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/hooks',
    'issue_events_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/events',
    'assignees_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/tags',
    'blobs_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/languages',
    'stargazers_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/stargazers',
    'contributors_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/contributors',
    'subscribers_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/subscribers',
    'subscription_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/subscription',
    'commits_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/merges',
    'archive_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/downloads',
    'issues_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/labels{/name}',
    'releases_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/AmmanChhetri/GiAN/deployments',
    'created_at': '2024-04-16T03:03:04Z',
    'updated_at': '2024-04-16T03:10:06Z',
    'pushed_at': '2024-04-16T03:07:43Z',
    'git_url': 'git://github.com/AmmanChhetri/GiAN.git',
    'ssh_url': 'git@github.com:AmmanChhetri/GiAN.git',
    'clone_url': 'https://github.com/AmmanChhetri/GiAN.git',
    'svn_url': 'https://github.com/AmmanChhetri/GiAN',
    'homepage': None,
    'size': 1,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': None,
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 706581265,
    'node_id': 'R_kgDOKh2TEQ',
    'name': 'MoodWave-backend',
    'full_name': 'AmmanChhetri/MoodWave-backend',
    'private': True,
    'owner': {
      'login': 'AmmanChhetri',
      'id': 121025542,
      'node_id': 'U_kgDOBza0Bg',
      'avatar_url': 'https://avatars.githubusercontent.com/u/121025542?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/AmmanChhetri',
      'html_url': 'https://github.com/AmmanChhetri',
      'followers_url': 'https://api.github.com/users/AmmanChhetri/followers',
      'following_url': 'https://api.github.com/users/AmmanChhetri/following{/other_user}',
      'gists_url': 'https://api.github.com/users/AmmanChhetri/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/AmmanChhetri/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/AmmanChhetri/subscriptions',
      'organizations_url': 'https://api.github.com/users/AmmanChhetri/orgs',
      'repos_url': 'https://api.github.com/users/AmmanChhetri/repos',
      'events_url': 'https://api.github.com/users/AmmanChhetri/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/AmmanChhetri/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/AmmanChhetri/MoodWave-backend',
    'description': "Moodwave backend API's",
    'fork': True,
    'url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend',
    'forks_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/forks',
    'keys_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/teams',
    'hooks_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/hooks',
    'issue_events_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/events',
    'assignees_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/tags',
    'blobs_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/languages',
    'stargazers_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/stargazers',
    'contributors_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/contributors',
    'subscribers_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/subscribers',
    'subscription_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/subscription',
    'commits_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/merges',
    'archive_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/downloads',
    'issues_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/labels{/name}',
    'releases_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/AmmanChhetri/MoodWave-backend/deployments',
    'created_at': '2023-10-18T08:26:34Z',
    'updated_at': '2023-10-18T08:26:34Z',
    'pushed_at': '2023-09-24T14:36:26Z',
    'git_url': 'git://github.com/AmmanChhetri/MoodWave-backend.git',
    'ssh_url': 'git@github.com:AmmanChhetri/MoodWave-backend.git',
    'clone_url': 'https://github.com/AmmanChhetri/MoodWave-backend.git',
    'svn_url': 'https://github.com/AmmanChhetri/MoodWave-backend',
    'homepage': None,
    'size': 21,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': None,
    'has_issues': False,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': False,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'mit',
      'name': 'MIT License',
      'spdx_id': 'MIT',
      'url': 'https://api.github.com/licenses/mit',
      'node_id': 'MDc6TGljZW5zZTEz'
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'private',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 617520041,
    'node_id': 'R_kgDOJM6bqQ',
    'name': 'SmartCalendar',
    'full_name': 'an33k25/SmartCalendar',
    'private': True,
    'owner': {
      'login': 'an33k25',
      'id': 99678188,
      'node_id': 'U_kgDOBfD37A',
      'avatar_url': 'https://avatars.githubusercontent.com/u/99678188?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/an33k25',
      'html_url': 'https://github.com/an33k25',
      'followers_url': 'https://api.github.com/users/an33k25/followers',
      'following_url': 'https://api.github.com/users/an33k25/following{/other_user}',
      'gists_url': 'https://api.github.com/users/an33k25/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/an33k25/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/an33k25/subscriptions',
      'organizations_url': 'https://api.github.com/users/an33k25/orgs',
      'repos_url': 'https://api.github.com/users/an33k25/repos',
      'events_url': 'https://api.github.com/users/an33k25/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/an33k25/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/an33k25/SmartCalendar',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/an33k25/SmartCalendar',
    'forks_url': 'https://api.github.com/repos/an33k25/SmartCalendar/forks',
    'keys_url': 'https://api.github.com/repos/an33k25/SmartCalendar/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/an33k25/SmartCalendar/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/an33k25/SmartCalendar/teams',
    'hooks_url': 'https://api.github.com/repos/an33k25/SmartCalendar/hooks',
    'issue_events_url': 'https://api.github.com/repos/an33k25/SmartCalendar/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/an33k25/SmartCalendar/events',
    'assignees_url': 'https://api.github.com/repos/an33k25/SmartCalendar/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/an33k25/SmartCalendar/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/an33k25/SmartCalendar/tags',
    'blobs_url': 'https://api.github.com/repos/an33k25/SmartCalendar/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/an33k25/SmartCalendar/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/an33k25/SmartCalendar/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/an33k25/SmartCalendar/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/an33k25/SmartCalendar/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/an33k25/SmartCalendar/languages',
    'stargazers_url': 'https://api.github.com/repos/an33k25/SmartCalendar/stargazers',
    'contributors_url': 'https://api.github.com/repos/an33k25/SmartCalendar/contributors',
    'subscribers_url': 'https://api.github.com/repos/an33k25/SmartCalendar/subscribers',
    'subscription_url': 'https://api.github.com/repos/an33k25/SmartCalendar/subscription',
    'commits_url': 'https://api.github.com/repos/an33k25/SmartCalendar/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/an33k25/SmartCalendar/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/an33k25/SmartCalendar/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/an33k25/SmartCalendar/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/an33k25/SmartCalendar/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/an33k25/SmartCalendar/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/an33k25/SmartCalendar/merges',
    'archive_url': 'https://api.github.com/repos/an33k25/SmartCalendar/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/an33k25/SmartCalendar/downloads',
    'issues_url': 'https://api.github.com/repos/an33k25/SmartCalendar/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/an33k25/SmartCalendar/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/an33k25/SmartCalendar/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/an33k25/SmartCalendar/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/an33k25/SmartCalendar/labels{/name}',
    'releases_url': 'https://api.github.com/repos/an33k25/SmartCalendar/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/an33k25/SmartCalendar/deployments',
    'created_at': '2023-03-22T15:01:15Z',
    'updated_at': '2023-03-22T15:01:15Z',
    'pushed_at': '2023-03-22T16:38:07Z',
    'git_url': 'git://github.com/an33k25/SmartCalendar.git',
    'ssh_url': 'git@github.com:an33k25/SmartCalendar.git',
    'clone_url': 'https://github.com/an33k25/SmartCalendar.git',
    'svn_url': 'https://github.com/an33k25/SmartCalendar',
    'homepage': None,
    'size': 4,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': None,
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': False,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 1,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'apache-2.0',
      'name': 'Apache License 2.0',
      'spdx_id': 'Apache-2.0',
      'url': 'https://api.github.com/licenses/apache-2.0',
      'node_id': 'MDc6TGljZW5zZTI='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'private',
    'forks': 1,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 713535788,
    'node_id': 'R_kgDOKoexLA',
    'name': 'mw-backend',
    'full_name': 'JERVISLOPES/mw-backend',
    'private': True,
    'owner': {
      'login': 'JERVISLOPES',
      'id': 129100794,
      'node_id': 'U_kgDOB7Hr-g',
      'avatar_url': 'https://avatars.githubusercontent.com/u/129100794?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/JERVISLOPES',
      'html_url': 'https://github.com/JERVISLOPES',
      'followers_url': 'https://api.github.com/users/JERVISLOPES/followers',
      'following_url': 'https://api.github.com/users/JERVISLOPES/following{/other_user}',
      'gists_url': 'https://api.github.com/users/JERVISLOPES/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/JERVISLOPES/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/JERVISLOPES/subscriptions',
      'organizations_url': 'https://api.github.com/users/JERVISLOPES/orgs',
      'repos_url': 'https://api.github.com/users/JERVISLOPES/repos',
      'events_url': 'https://api.github.com/users/JERVISLOPES/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/JERVISLOPES/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/JERVISLOPES/mw-backend',
    'description': "Moodwave backend API's",
    'fork': True,
    'url': 'https://api.github.com/repos/JERVISLOPES/mw-backend',
    'forks_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/forks',
    'keys_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/teams',
    'hooks_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/hooks',
    'issue_events_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/events',
    'assignees_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/tags',
    'blobs_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/languages',
    'stargazers_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/stargazers',
    'contributors_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/contributors',
    'subscribers_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/subscribers',
    'subscription_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/subscription',
    'commits_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/merges',
    'archive_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/downloads',
    'issues_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/labels{/name}',
    'releases_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/JERVISLOPES/mw-backend/deployments',
    'created_at': '2023-11-02T18:06:11Z',
    'updated_at': '2023-11-02T18:06:11Z',
    'pushed_at': '2023-11-01T12:46:04Z',
    'git_url': 'git://github.com/JERVISLOPES/mw-backend.git',
    'ssh_url': 'git@github.com:JERVISLOPES/mw-backend.git',
    'clone_url': 'https://github.com/JERVISLOPES/mw-backend.git',
    'svn_url': 'https://github.com/JERVISLOPES/mw-backend',
    'homepage': None,
    'size': 42739,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': None,
    'has_issues': False,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': False,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'mit',
      'name': 'MIT License',
      'spdx_id': 'MIT',
      'url': 'https://api.github.com/licenses/mit',
      'node_id': 'MDc6TGljZW5zZTEz'
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'private',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 588651484,
    'node_id': 'R_kgDOIxYb3A',
    'name': 'algorithms',
    'full_name': 'justforfunweb/algorithms',
    'private': False,
    'owner': {
      'login': 'justforfunweb',
      'id': 113769799,
      'node_id': 'U_kgDOBsf9Rw',
      'avatar_url': 'https://avatars.githubusercontent.com/u/113769799?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/justforfunweb',
      'html_url': 'https://github.com/justforfunweb',
      'followers_url': 'https://api.github.com/users/justforfunweb/followers',
      'following_url': 'https://api.github.com/users/justforfunweb/following{/other_user}',
      'gists_url': 'https://api.github.com/users/justforfunweb/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/justforfunweb/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/justforfunweb/subscriptions',
      'organizations_url': 'https://api.github.com/users/justforfunweb/orgs',
      'repos_url': 'https://api.github.com/users/justforfunweb/repos',
      'events_url': 'https://api.github.com/users/justforfunweb/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/justforfunweb/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/justforfunweb/algorithms',
    'description': 'Basic Algorithms in C, C++, Python',
    'fork': False,
    'url': 'https://api.github.com/repos/justforfunweb/algorithms',
    'forks_url': 'https://api.github.com/repos/justforfunweb/algorithms/forks',
    'keys_url': 'https://api.github.com/repos/justforfunweb/algorithms/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/justforfunweb/algorithms/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/justforfunweb/algorithms/teams',
    'hooks_url': 'https://api.github.com/repos/justforfunweb/algorithms/hooks',
    'issue_events_url': 'https://api.github.com/repos/justforfunweb/algorithms/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/justforfunweb/algorithms/events',
    'assignees_url': 'https://api.github.com/repos/justforfunweb/algorithms/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/justforfunweb/algorithms/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/justforfunweb/algorithms/tags',
    'blobs_url': 'https://api.github.com/repos/justforfunweb/algorithms/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/justforfunweb/algorithms/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/justforfunweb/algorithms/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/justforfunweb/algorithms/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/justforfunweb/algorithms/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/justforfunweb/algorithms/languages',
    'stargazers_url': 'https://api.github.com/repos/justforfunweb/algorithms/stargazers',
    'contributors_url': 'https://api.github.com/repos/justforfunweb/algorithms/contributors',
    'subscribers_url': 'https://api.github.com/repos/justforfunweb/algorithms/subscribers',
    'subscription_url': 'https://api.github.com/repos/justforfunweb/algorithms/subscription',
    'commits_url': 'https://api.github.com/repos/justforfunweb/algorithms/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/justforfunweb/algorithms/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/justforfunweb/algorithms/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/justforfunweb/algorithms/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/justforfunweb/algorithms/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/justforfunweb/algorithms/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/justforfunweb/algorithms/merges',
    'archive_url': 'https://api.github.com/repos/justforfunweb/algorithms/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/justforfunweb/algorithms/downloads',
    'issues_url': 'https://api.github.com/repos/justforfunweb/algorithms/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/justforfunweb/algorithms/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/justforfunweb/algorithms/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/justforfunweb/algorithms/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/justforfunweb/algorithms/labels{/name}',
    'releases_url': 'https://api.github.com/repos/justforfunweb/algorithms/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/justforfunweb/algorithms/deployments',
    'created_at': '2023-01-13T16:42:28Z',
    'updated_at': '2023-03-04T18:16:58Z',
    'pushed_at': '2023-01-17T08:45:25Z',
    'git_url': 'git://github.com/justforfunweb/algorithms.git',
    'ssh_url': 'git@github.com:justforfunweb/algorithms.git',
    'clone_url': 'https://github.com/justforfunweb/algorithms.git',
    'svn_url': 'https://github.com/justforfunweb/algorithms',
    'homepage': None,
    'size': 29,
    'stargazers_count': 2,
    'watchers_count': 2,
    'language': 'C',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 2,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 735122127,
    'node_id': 'R_kgDOK9ESzw',
    'name': 'moodwave-backend',
    'full_name': 'moodwave/moodwave-backend',
    'private': True,
    'owner': {
      'login': 'moodwave',
      'id': 144619929,
      'node_id': 'U_kgDOCJ65mQ',
      'avatar_url': 'https://avatars.githubusercontent.com/u/144619929?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/moodwave',
      'html_url': 'https://github.com/moodwave',
      'followers_url': 'https://api.github.com/users/moodwave/followers',
      'following_url': 'https://api.github.com/users/moodwave/following{/other_user}',
      'gists_url': 'https://api.github.com/users/moodwave/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/moodwave/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/moodwave/subscriptions',
      'organizations_url': 'https://api.github.com/users/moodwave/orgs',
      'repos_url': 'https://api.github.com/users/moodwave/repos',
      'events_url': 'https://api.github.com/users/moodwave/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/moodwave/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/moodwave/moodwave-backend',
    'description': 'Official deployed moodwave backend',
    'fork': False,
    'url': 'https://api.github.com/repos/moodwave/moodwave-backend',
    'forks_url': 'https://api.github.com/repos/moodwave/moodwave-backend/forks',
    'keys_url': 'https://api.github.com/repos/moodwave/moodwave-backend/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/moodwave/moodwave-backend/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/moodwave/moodwave-backend/teams',
    'hooks_url': 'https://api.github.com/repos/moodwave/moodwave-backend/hooks',
    'issue_events_url': 'https://api.github.com/repos/moodwave/moodwave-backend/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/moodwave/moodwave-backend/events',
    'assignees_url': 'https://api.github.com/repos/moodwave/moodwave-backend/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/moodwave/moodwave-backend/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/moodwave/moodwave-backend/tags',
    'blobs_url': 'https://api.github.com/repos/moodwave/moodwave-backend/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/moodwave/moodwave-backend/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/moodwave/moodwave-backend/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/moodwave/moodwave-backend/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/moodwave/moodwave-backend/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/moodwave/moodwave-backend/languages',
    'stargazers_url': 'https://api.github.com/repos/moodwave/moodwave-backend/stargazers',
    'contributors_url': 'https://api.github.com/repos/moodwave/moodwave-backend/contributors',
    'subscribers_url': 'https://api.github.com/repos/moodwave/moodwave-backend/subscribers',
    'subscription_url': 'https://api.github.com/repos/moodwave/moodwave-backend/subscription',
    'commits_url': 'https://api.github.com/repos/moodwave/moodwave-backend/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/moodwave/moodwave-backend/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/moodwave/moodwave-backend/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/moodwave/moodwave-backend/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/moodwave/moodwave-backend/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/moodwave/moodwave-backend/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/moodwave/moodwave-backend/merges',
    'archive_url': 'https://api.github.com/repos/moodwave/moodwave-backend/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/moodwave/moodwave-backend/downloads',
    'issues_url': 'https://api.github.com/repos/moodwave/moodwave-backend/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/moodwave/moodwave-backend/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/moodwave/moodwave-backend/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/moodwave/moodwave-backend/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/moodwave/moodwave-backend/labels{/name}',
    'releases_url': 'https://api.github.com/repos/moodwave/moodwave-backend/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/moodwave/moodwave-backend/deployments',
    'created_at': '2023-12-23T18:34:51Z',
    'updated_at': '2023-12-24T15:05:36Z',
    'pushed_at': '2023-12-25T05:42:29Z',
    'git_url': 'git://github.com/moodwave/moodwave-backend.git',
    'ssh_url': 'git@github.com:moodwave/moodwave-backend.git',
    'clone_url': 'https://github.com/moodwave/moodwave-backend.git',
    'svn_url': 'https://github.com/moodwave/moodwave-backend',
    'homepage': None,
    'size': 20,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': 'Python',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': False,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 1,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'private',
    'forks': 1,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 689722908,
    'node_id': 'R_kgDOKRxWHA',
    'name': 'moodwave.github.io',
    'full_name': 'moodwave/moodwave.github.io',
    'private': False,
    'owner': {
      'login': 'moodwave',
      'id': 144619929,
      'node_id': 'U_kgDOCJ65mQ',
      'avatar_url': 'https://avatars.githubusercontent.com/u/144619929?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/moodwave',
      'html_url': 'https://github.com/moodwave',
      'followers_url': 'https://api.github.com/users/moodwave/followers',
      'following_url': 'https://api.github.com/users/moodwave/following{/other_user}',
      'gists_url': 'https://api.github.com/users/moodwave/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/moodwave/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/moodwave/subscriptions',
      'organizations_url': 'https://api.github.com/users/moodwave/orgs',
      'repos_url': 'https://api.github.com/users/moodwave/repos',
      'events_url': 'https://api.github.com/users/moodwave/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/moodwave/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/moodwave/moodwave.github.io',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/moodwave/moodwave.github.io',
    'forks_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/forks',
    'keys_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/teams',
    'hooks_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/hooks',
    'issue_events_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/events',
    'assignees_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/tags',
    'blobs_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/languages',
    'stargazers_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/stargazers',
    'contributors_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/contributors',
    'subscribers_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/subscribers',
    'subscription_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/subscription',
    'commits_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/merges',
    'archive_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/downloads',
    'issues_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/labels{/name}',
    'releases_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/moodwave/moodwave.github.io/deployments',
    'created_at': '2023-09-10T17:48:28Z',
    'updated_at': '2023-09-10T18:22:27Z',
    'pushed_at': '2023-10-12T07:11:51Z',
    'git_url': 'git://github.com/moodwave/moodwave.github.io.git',
    'ssh_url': 'git@github.com:moodwave/moodwave.github.io.git',
    'clone_url': 'https://github.com/moodwave/moodwave.github.io.git',
    'svn_url': 'https://github.com/moodwave/moodwave.github.io',
    'homepage': None,
    'size': 16,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': 'HTML',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': True,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'mit',
      'name': 'MIT License',
      'spdx_id': 'MIT',
      'url': 'https://api.github.com/licenses/mit',
      'node_id': 'MDc6TGljZW5zZTEz'
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 694768658,
    'node_id': 'R_kgDOKWlUEg',
    'name': 'mw-backend',
    'full_name': 'moodwave/mw-backend',
    'private': True,
    'owner': {
      'login': 'moodwave',
      'id': 144619929,
      'node_id': 'U_kgDOCJ65mQ',
      'avatar_url': 'https://avatars.githubusercontent.com/u/144619929?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/moodwave',
      'html_url': 'https://github.com/moodwave',
      'followers_url': 'https://api.github.com/users/moodwave/followers',
      'following_url': 'https://api.github.com/users/moodwave/following{/other_user}',
      'gists_url': 'https://api.github.com/users/moodwave/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/moodwave/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/moodwave/subscriptions',
      'organizations_url': 'https://api.github.com/users/moodwave/orgs',
      'repos_url': 'https://api.github.com/users/moodwave/repos',
      'events_url': 'https://api.github.com/users/moodwave/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/moodwave/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/moodwave/mw-backend',
    'description': "Moodwave backend API's",
    'fork': False,
    'url': 'https://api.github.com/repos/moodwave/mw-backend',
    'forks_url': 'https://api.github.com/repos/moodwave/mw-backend/forks',
    'keys_url': 'https://api.github.com/repos/moodwave/mw-backend/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/moodwave/mw-backend/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/moodwave/mw-backend/teams',
    'hooks_url': 'https://api.github.com/repos/moodwave/mw-backend/hooks',
    'issue_events_url': 'https://api.github.com/repos/moodwave/mw-backend/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/moodwave/mw-backend/events',
    'assignees_url': 'https://api.github.com/repos/moodwave/mw-backend/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/moodwave/mw-backend/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/moodwave/mw-backend/tags',
    'blobs_url': 'https://api.github.com/repos/moodwave/mw-backend/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/moodwave/mw-backend/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/moodwave/mw-backend/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/moodwave/mw-backend/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/moodwave/mw-backend/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/moodwave/mw-backend/languages',
    'stargazers_url': 'https://api.github.com/repos/moodwave/mw-backend/stargazers',
    'contributors_url': 'https://api.github.com/repos/moodwave/mw-backend/contributors',
    'subscribers_url': 'https://api.github.com/repos/moodwave/mw-backend/subscribers',
    'subscription_url': 'https://api.github.com/repos/moodwave/mw-backend/subscription',
    'commits_url': 'https://api.github.com/repos/moodwave/mw-backend/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/moodwave/mw-backend/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/moodwave/mw-backend/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/moodwave/mw-backend/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/moodwave/mw-backend/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/moodwave/mw-backend/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/moodwave/mw-backend/merges',
    'archive_url': 'https://api.github.com/repos/moodwave/mw-backend/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/moodwave/mw-backend/downloads',
    'issues_url': 'https://api.github.com/repos/moodwave/mw-backend/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/moodwave/mw-backend/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/moodwave/mw-backend/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/moodwave/mw-backend/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/moodwave/mw-backend/labels{/name}',
    'releases_url': 'https://api.github.com/repos/moodwave/mw-backend/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/moodwave/mw-backend/deployments',
    'created_at': '2023-09-21T16:47:59Z',
    'updated_at': '2023-09-21T20:04:42Z',
    'pushed_at': '2023-12-14T14:45:14Z',
    'git_url': 'git://github.com/moodwave/mw-backend.git',
    'ssh_url': 'git@github.com:moodwave/mw-backend.git',
    'clone_url': 'https://github.com/moodwave/mw-backend.git',
    'svn_url': 'https://github.com/moodwave/mw-backend',
    'homepage': None,
    'size': 42757,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': 'Python',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': False,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 2,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'mit',
      'name': 'MIT License',
      'spdx_id': 'MIT',
      'url': 'https://api.github.com/licenses/mit',
      'node_id': 'MDc6TGljZW5zZTEz'
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'private',
    'forks': 2,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': False,
      'maintain': False,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 773024681,
    'node_id': 'R_kgDOLhNrqQ',
    'name': 'blimp-total',
    'full_name': 'shubanms/blimp-total',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/blimp-total',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/blimp-total',
    'forks_url': 'https://api.github.com/repos/shubanms/blimp-total/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/blimp-total/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/blimp-total/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/blimp-total/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/blimp-total/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/blimp-total/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/blimp-total/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/blimp-total/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/blimp-total/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/blimp-total/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/blimp-total/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/blimp-total/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/blimp-total/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/blimp-total/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/blimp-total/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/blimp-total/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/blimp-total/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/blimp-total/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/blimp-total/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/blimp-total/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/blimp-total/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/blimp-total/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/blimp-total/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/blimp-total/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/blimp-total/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/blimp-total/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/blimp-total/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/blimp-total/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/blimp-total/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/blimp-total/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/blimp-total/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/blimp-total/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/blimp-total/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/blimp-total/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/blimp-total/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/blimp-total/deployments',
    'created_at': '2024-03-16T14:35:43Z',
    'updated_at': '2024-03-17T17:57:24Z',
    'pushed_at': '2024-03-16T14:35:43Z',
    'git_url': 'git://github.com/shubanms/blimp-total.git',
    'ssh_url': 'git@github.com:shubanms/blimp-total.git',
    'clone_url': 'https://github.com/shubanms/blimp-total.git',
    'svn_url': 'https://github.com/shubanms/blimp-total',
    'homepage': None,
    'size': 14,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': None,
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'gpl-3.0',
      'name': 'GNU General Public License v3.0',
      'spdx_id': 'GPL-3.0',
      'url': 'https://api.github.com/licenses/gpl-3.0',
      'node_id': 'MDc6TGljZW5zZTk='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 579473406,
    'node_id': 'R_kgDOIooP_g',
    'name': 'CloudNine',
    'full_name': 'shubanms/CloudNine',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/CloudNine',
    'description': 'This is a repository for the GCCP Project',
    'fork': True,
    'url': 'https://api.github.com/repos/shubanms/CloudNine',
    'forks_url': 'https://api.github.com/repos/shubanms/CloudNine/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/CloudNine/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/CloudNine/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/CloudNine/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/CloudNine/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/CloudNine/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/CloudNine/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/CloudNine/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/CloudNine/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/CloudNine/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/CloudNine/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/CloudNine/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/CloudNine/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/CloudNine/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/CloudNine/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/CloudNine/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/CloudNine/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/CloudNine/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/CloudNine/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/CloudNine/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/CloudNine/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/CloudNine/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/CloudNine/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/CloudNine/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/CloudNine/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/CloudNine/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/CloudNine/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/CloudNine/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/CloudNine/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/CloudNine/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/CloudNine/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/CloudNine/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/CloudNine/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/CloudNine/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/CloudNine/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/CloudNine/deployments',
    'created_at': '2022-12-17T20:13:57Z',
    'updated_at': '2023-01-24T13:13:47Z',
    'pushed_at': '2023-01-08T15:01:00Z',
    'git_url': 'git://github.com/shubanms/CloudNine.git',
    'ssh_url': 'git@github.com:shubanms/CloudNine.git',
    'clone_url': 'https://github.com/shubanms/CloudNine.git',
    'svn_url': 'https://github.com/shubanms/CloudNine',
    'homepage': '',
    'size': 185,
    'stargazers_count': 3,
    'watchers_count': 3,
    'language': 'HTML',
    'has_issues': False,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 3,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 580891393,
    'node_id': 'R_kgDOIp-zAQ',
    'name': 'Co2-emissions',
    'full_name': 'shubanms/Co2-emissions',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Co2-emissions',
    'description': 'R program and respective libraries used for a project on the analysis of CO2 emissions under various factors using statistical tools such as linear regression and descriptive statistics',
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/Co2-emissions',
    'forks_url': 'https://api.github.com/repos/shubanms/Co2-emissions/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Co2-emissions/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Co2-emissions/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Co2-emissions/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Co2-emissions/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Co2-emissions/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Co2-emissions/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Co2-emissions/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Co2-emissions/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Co2-emissions/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Co2-emissions/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Co2-emissions/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Co2-emissions/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Co2-emissions/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Co2-emissions/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Co2-emissions/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Co2-emissions/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Co2-emissions/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Co2-emissions/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Co2-emissions/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Co2-emissions/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Co2-emissions/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Co2-emissions/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Co2-emissions/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Co2-emissions/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Co2-emissions/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Co2-emissions/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Co2-emissions/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Co2-emissions/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Co2-emissions/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Co2-emissions/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Co2-emissions/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Co2-emissions/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Co2-emissions/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Co2-emissions/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Co2-emissions/deployments',
    'created_at': '2022-12-21T18:04:35Z',
    'updated_at': '2023-01-31T22:28:43Z',
    'pushed_at': '2023-01-08T06:44:49Z',
    'git_url': 'git://github.com/shubanms/Co2-emissions.git',
    'ssh_url': 'git@github.com:shubanms/Co2-emissions.git',
    'clone_url': 'https://github.com/shubanms/Co2-emissions.git',
    'svn_url': 'https://github.com/shubanms/Co2-emissions',
    'homepage': '',
    'size': 3214,
    'stargazers_count': 3,
    'watchers_count': 3,
    'language': 'R',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      'analysis',
      'analytics',
      'data',
      'data-science',
      'r'
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 3,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 664232861,
    'node_id': 'R_kgDOJ5djnQ',
    'name': 'Crop-Prediction',
    'full_name': 'shubanms/Crop-Prediction',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Crop-Prediction',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/Crop-Prediction',
    'forks_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Crop-Prediction/deployments',
    'created_at': '2023-07-09T10:29:31Z',
    'updated_at': '2023-07-11T17:03:07Z',
    'pushed_at': '2023-07-13T03:06:34Z',
    'git_url': 'git://github.com/shubanms/Crop-Prediction.git',
    'ssh_url': 'git@github.com:shubanms/Crop-Prediction.git',
    'clone_url': 'https://github.com/shubanms/Crop-Prediction.git',
    'svn_url': 'https://github.com/shubanms/Crop-Prediction',
    'homepage': None,
    'size': 1162,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': 'Jupyter Notebook',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'mit',
      'name': 'MIT License',
      'spdx_id': 'MIT',
      'url': 'https://api.github.com/licenses/mit',
      'node_id': 'MDc6TGljZW5zZTEz'
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 773458015,
    'node_id': 'R_kgDOLhoIXw',
    'name': 'Deep-Learning',
    'full_name': 'shubanms/Deep-Learning',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Deep-Learning',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/Deep-Learning',
    'forks_url': 'https://api.github.com/repos/shubanms/Deep-Learning/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Deep-Learning/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Deep-Learning/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Deep-Learning/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Deep-Learning/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Deep-Learning/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Deep-Learning/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Deep-Learning/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Deep-Learning/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Deep-Learning/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Deep-Learning/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Deep-Learning/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Deep-Learning/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Deep-Learning/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Deep-Learning/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Deep-Learning/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Deep-Learning/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Deep-Learning/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Deep-Learning/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Deep-Learning/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Deep-Learning/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Deep-Learning/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Deep-Learning/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Deep-Learning/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Deep-Learning/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Deep-Learning/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Deep-Learning/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Deep-Learning/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Deep-Learning/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Deep-Learning/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Deep-Learning/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Deep-Learning/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Deep-Learning/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Deep-Learning/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Deep-Learning/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Deep-Learning/deployments',
    'created_at': '2024-03-17T18:02:45Z',
    'updated_at': '2024-03-17T18:02:46Z',
    'pushed_at': '2024-03-17T18:02:46Z',
    'git_url': 'git://github.com/shubanms/Deep-Learning.git',
    'ssh_url': 'git@github.com:shubanms/Deep-Learning.git',
    'clone_url': 'https://github.com/shubanms/Deep-Learning.git',
    'svn_url': 'https://github.com/shubanms/Deep-Learning',
    'homepage': None,
    'size': 14,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': None,
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'gpl-3.0',
      'name': 'GNU General Public License v3.0',
      'spdx_id': 'GPL-3.0',
      'url': 'https://api.github.com/licenses/gpl-3.0',
      'node_id': 'MDc6TGljZW5zZTk='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 750873589,
    'node_id': 'R_kgDOLMFr9Q',
    'name': 'DL-Azure-Deployment',
    'full_name': 'shubanms/DL-Azure-Deployment',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/DL-Azure-Deployment',
    'description': 'Code to deploy and test DL model from Azure',
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment',
    'forks_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/DL-Azure-Deployment/deployments',
    'created_at': '2024-01-31T13:50:05Z',
    'updated_at': '2024-02-01T10:26:36Z',
    'pushed_at': '2024-02-01T18:42:17Z',
    'git_url': 'git://github.com/shubanms/DL-Azure-Deployment.git',
    'ssh_url': 'git@github.com:shubanms/DL-Azure-Deployment.git',
    'clone_url': 'https://github.com/shubanms/DL-Azure-Deployment.git',
    'svn_url': 'https://github.com/shubanms/DL-Azure-Deployment',
    'homepage': '',
    'size': 3886,
    'stargazers_count': 2,
    'watchers_count': 2,
    'language': 'Jupyter Notebook',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 4,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'mit',
      'name': 'MIT License',
      'spdx_id': 'MIT',
      'url': 'https://api.github.com/licenses/mit',
      'node_id': 'MDc6TGljZW5zZTEz'
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 4,
    'open_issues': 0,
    'watchers': 2,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 773011513,
    'node_id': 'R_kgDOLhM4OQ',
    'name': 'Dry-Beans',
    'full_name': 'shubanms/Dry-Beans',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Dry-Beans',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/Dry-Beans',
    'forks_url': 'https://api.github.com/repos/shubanms/Dry-Beans/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Dry-Beans/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Dry-Beans/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Dry-Beans/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Dry-Beans/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Dry-Beans/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Dry-Beans/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Dry-Beans/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Dry-Beans/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Dry-Beans/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Dry-Beans/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Dry-Beans/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Dry-Beans/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Dry-Beans/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Dry-Beans/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Dry-Beans/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Dry-Beans/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Dry-Beans/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Dry-Beans/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Dry-Beans/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Dry-Beans/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Dry-Beans/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Dry-Beans/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Dry-Beans/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Dry-Beans/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Dry-Beans/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Dry-Beans/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Dry-Beans/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Dry-Beans/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Dry-Beans/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Dry-Beans/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Dry-Beans/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Dry-Beans/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Dry-Beans/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Dry-Beans/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Dry-Beans/deployments',
    'created_at': '2024-03-16T13:54:07Z',
    'updated_at': '2024-03-17T17:57:24Z',
    'pushed_at': '2024-03-16T13:54:07Z',
    'git_url': 'git://github.com/shubanms/Dry-Beans.git',
    'ssh_url': 'git@github.com:shubanms/Dry-Beans.git',
    'clone_url': 'https://github.com/shubanms/Dry-Beans.git',
    'svn_url': 'https://github.com/shubanms/Dry-Beans',
    'homepage': None,
    'size': 16,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': None,
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'gpl-3.0',
      'name': 'GNU General Public License v3.0',
      'spdx_id': 'GPL-3.0',
      'url': 'https://api.github.com/licenses/gpl-3.0',
      'node_id': 'MDc6TGljZW5zZTk='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 767061527,
    'node_id': 'R_kgDOLbhuFw',
    'name': 'DSA',
    'full_name': 'shubanms/DSA',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/DSA',
    'description': 'DSA from the Take U Forward Course',
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/DSA',
    'forks_url': 'https://api.github.com/repos/shubanms/DSA/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/DSA/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/DSA/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/DSA/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/DSA/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/DSA/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/DSA/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/DSA/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/DSA/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/DSA/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/DSA/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/DSA/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/DSA/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/DSA/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/DSA/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/DSA/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/DSA/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/DSA/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/DSA/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/DSA/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/DSA/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/DSA/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/DSA/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/DSA/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/DSA/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/DSA/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/DSA/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/DSA/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/DSA/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/DSA/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/DSA/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/DSA/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/DSA/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/DSA/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/DSA/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/DSA/deployments',
    'created_at': '2024-03-04T16:27:56Z',
    'updated_at': '2024-03-17T17:57:27Z',
    'pushed_at': '2024-03-08T17:11:17Z',
    'git_url': 'git://github.com/shubanms/DSA.git',
    'ssh_url': 'git@github.com:shubanms/DSA.git',
    'clone_url': 'https://github.com/shubanms/DSA.git',
    'svn_url': 'https://github.com/shubanms/DSA',
    'homepage': None,
    'size': 7,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': 'Jupyter Notebook',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'master',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 707360760,
    'node_id': 'R_kgDOKil3-A',
    'name': 'FERPlus',
    'full_name': 'shubanms/FERPlus',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/FERPlus',
    'description': 'This is the FER+ new label annotations for the Emotion FER dataset.',
    'fork': True,
    'url': 'https://api.github.com/repos/shubanms/FERPlus',
    'forks_url': 'https://api.github.com/repos/shubanms/FERPlus/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/FERPlus/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/FERPlus/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/FERPlus/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/FERPlus/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/FERPlus/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/FERPlus/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/FERPlus/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/FERPlus/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/FERPlus/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/FERPlus/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/FERPlus/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/FERPlus/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/FERPlus/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/FERPlus/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/FERPlus/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/FERPlus/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/FERPlus/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/FERPlus/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/FERPlus/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/FERPlus/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/FERPlus/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/FERPlus/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/FERPlus/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/FERPlus/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/FERPlus/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/FERPlus/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/FERPlus/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/FERPlus/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/FERPlus/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/FERPlus/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/FERPlus/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/FERPlus/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/FERPlus/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/FERPlus/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/FERPlus/deployments',
    'created_at': '2023-10-19T18:27:38Z',
    'updated_at': '2024-03-17T17:57:38Z',
    'pushed_at': '2023-06-12T20:52:53Z',
    'git_url': 'git://github.com/shubanms/FERPlus.git',
    'ssh_url': 'git@github.com:shubanms/FERPlus.git',
    'clone_url': 'https://github.com/shubanms/FERPlus.git',
    'svn_url': 'https://github.com/shubanms/FERPlus',
    'homepage': 'https://arxiv.org/abs/1608.01041',
    'size': 653,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': None,
    'has_issues': False,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'other',
      'name': 'Other',
      'spdx_id': 'NOASSERTION',
      'url': None,
      'node_id': 'MDc6TGljZW5zZTA='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'master',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 663697743,
    'node_id': 'R_kgDOJ485Tw',
    'name': 'Food-Management-Systems',
    'full_name': 'shubanms/Food-Management-Systems',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Food-Management-Systems',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/Food-Management-Systems',
    'forks_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Food-Management-Systems/deployments',
    'created_at': '2023-07-07T22:43:02Z',
    'updated_at': '2023-07-07T23:13:43Z',
    'pushed_at': '2024-01-17T15:46:14Z',
    'git_url': 'git://github.com/shubanms/Food-Management-Systems.git',
    'ssh_url': 'git@github.com:shubanms/Food-Management-Systems.git',
    'clone_url': 'https://github.com/shubanms/Food-Management-Systems.git',
    'svn_url': 'https://github.com/shubanms/Food-Management-Systems',
    'homepage': None,
    'size': 23,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': 'Jupyter Notebook',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'apache-2.0',
      'name': 'Apache License 2.0',
      'spdx_id': 'Apache-2.0',
      'url': 'https://api.github.com/licenses/apache-2.0',
      'node_id': 'MDc6TGljZW5zZTI='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 624757816,
    'node_id': 'R_kgDOJT0MOA',
    'name': 'GDSC-DeathClock',
    'full_name': 'shubanms/GDSC-DeathClock',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/GDSC-DeathClock',
    'description': None,
    'fork': True,
    'url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock',
    'forks_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/GDSC-DeathClock/deployments',
    'created_at': '2023-04-07T07:26:43Z',
    'updated_at': '2023-04-08T06:37:43Z',
    'pushed_at': '2023-04-07T05:43:29Z',
    'git_url': 'git://github.com/shubanms/GDSC-DeathClock.git',
    'ssh_url': 'git@github.com:shubanms/GDSC-DeathClock.git',
    'clone_url': 'https://github.com/shubanms/GDSC-DeathClock.git',
    'svn_url': 'https://github.com/shubanms/GDSC-DeathClock',
    'homepage': None,
    'size': 152,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': None,
    'has_issues': False,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 624757758,
    'node_id': 'R_kgDOJT0L_g',
    'name': 'gdsc-wow-hackathon',
    'full_name': 'shubanms/gdsc-wow-hackathon',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/gdsc-wow-hackathon',
    'description': 'Repository for Death Clock, Hackathon event in GDSC WOW',
    'fork': True,
    'url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon',
    'forks_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/gdsc-wow-hackathon/deployments',
    'created_at': '2023-04-07T07:26:34Z',
    'updated_at': '2023-04-08T06:37:34Z',
    'pushed_at': '2023-04-08T06:48:49Z',
    'git_url': 'git://github.com/shubanms/gdsc-wow-hackathon.git',
    'ssh_url': 'git@github.com:shubanms/gdsc-wow-hackathon.git',
    'clone_url': 'https://github.com/shubanms/gdsc-wow-hackathon.git',
    'svn_url': 'https://github.com/shubanms/gdsc-wow-hackathon',
    'homepage': '',
    'size': 14352,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': 'Jupyter Notebook',
    'has_issues': False,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'master',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 747262160,
    'node_id': 'R_kgDOLIpQ0A',
    'name': 'Geaze',
    'full_name': 'shubanms/Geaze',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Geaze',
    'description': 'Geaze is a cutting-edge cross-chain wallet implementation that empowers users with the ability to perform gasless transactions across various EVM-compatible blockchains.',
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/Geaze',
    'forks_url': 'https://api.github.com/repos/shubanms/Geaze/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Geaze/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Geaze/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Geaze/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Geaze/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Geaze/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Geaze/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Geaze/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Geaze/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Geaze/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Geaze/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Geaze/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Geaze/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Geaze/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Geaze/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Geaze/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Geaze/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Geaze/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Geaze/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Geaze/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Geaze/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Geaze/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Geaze/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Geaze/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Geaze/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Geaze/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Geaze/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Geaze/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Geaze/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Geaze/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Geaze/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Geaze/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Geaze/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Geaze/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Geaze/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Geaze/deployments',
    'created_at': '2024-01-23T15:35:01Z',
    'updated_at': '2024-03-17T17:57:27Z',
    'pushed_at': '2024-01-24T04:40:15Z',
    'git_url': 'git://github.com/shubanms/Geaze.git',
    'ssh_url': 'git@github.com:shubanms/Geaze.git',
    'clone_url': 'https://github.com/shubanms/Geaze.git',
    'svn_url': 'https://github.com/shubanms/Geaze',
    'homepage': None,
    'size': 6,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': 'Python',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'mit',
      'name': 'MIT License',
      'spdx_id': 'MIT',
      'url': 'https://api.github.com/licenses/mit',
      'node_id': 'MDc6TGljZW5zZTEz'
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 787177368,
    'node_id': 'R_kgDOLutfmA',
    'name': 'GiAN-backend',
    'full_name': 'shubanms/GiAN-backend',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/GiAN-backend',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/GiAN-backend',
    'forks_url': 'https://api.github.com/repos/shubanms/GiAN-backend/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/GiAN-backend/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/GiAN-backend/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/GiAN-backend/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/GiAN-backend/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/GiAN-backend/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/GiAN-backend/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/GiAN-backend/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/GiAN-backend/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/GiAN-backend/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/GiAN-backend/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/GiAN-backend/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/GiAN-backend/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/GiAN-backend/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/GiAN-backend/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/GiAN-backend/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/GiAN-backend/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/GiAN-backend/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/GiAN-backend/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/GiAN-backend/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/GiAN-backend/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/GiAN-backend/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/GiAN-backend/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/GiAN-backend/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/GiAN-backend/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/GiAN-backend/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/GiAN-backend/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/GiAN-backend/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/GiAN-backend/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/GiAN-backend/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/GiAN-backend/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/GiAN-backend/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/GiAN-backend/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/GiAN-backend/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/GiAN-backend/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/GiAN-backend/deployments',
    'created_at': '2024-04-16T03:10:46Z',
    'updated_at': '2024-04-16T04:44:41Z',
    'pushed_at': '2024-04-16T04:44:37Z',
    'git_url': 'git://github.com/shubanms/GiAN-backend.git',
    'ssh_url': 'git@github.com:shubanms/GiAN-backend.git',
    'clone_url': 'https://github.com/shubanms/GiAN-backend.git',
    'svn_url': 'https://github.com/shubanms/GiAN-backend',
    'homepage': None,
    'size': 4,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': 'Jupyter Notebook',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 621883406,
    'node_id': 'R_kgDOJREwDg',
    'name': 'HART',
    'full_name': 'shubanms/HART',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/HART',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/HART',
    'forks_url': 'https://api.github.com/repos/shubanms/HART/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/HART/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/HART/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/HART/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/HART/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/HART/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/HART/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/HART/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/HART/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/HART/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/HART/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/HART/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/HART/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/HART/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/HART/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/HART/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/HART/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/HART/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/HART/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/HART/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/HART/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/HART/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/HART/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/HART/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/HART/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/HART/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/HART/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/HART/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/HART/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/HART/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/HART/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/HART/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/HART/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/HART/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/HART/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/HART/deployments',
    'created_at': '2023-03-31T15:37:49Z',
    'updated_at': '2023-04-05T14:10:56Z',
    'pushed_at': '2023-03-31T15:40:17Z',
    'git_url': 'git://github.com/shubanms/HART.git',
    'ssh_url': 'git@github.com:shubanms/HART.git',
    'clone_url': 'https://github.com/shubanms/HART.git',
    'svn_url': 'https://github.com/shubanms/HART',
    'homepage': None,
    'size': 259,
    'stargazers_count': 3,
    'watchers_count': 3,
    'language': 'Jupyter Notebook',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'apache-2.0',
      'name': 'Apache License 2.0',
      'spdx_id': 'Apache-2.0',
      'url': 'https://api.github.com/licenses/apache-2.0',
      'node_id': 'MDc6TGljZW5zZTI='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 3,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 581283153,
    'node_id': 'R_kgDOIqWtUQ',
    'name': 'Java-snake-game',
    'full_name': 'shubanms/Java-snake-game',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Java-snake-game',
    'description': 'A retro snake arcade game in Java',
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/Java-snake-game',
    'forks_url': 'https://api.github.com/repos/shubanms/Java-snake-game/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Java-snake-game/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Java-snake-game/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Java-snake-game/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Java-snake-game/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Java-snake-game/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Java-snake-game/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Java-snake-game/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Java-snake-game/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Java-snake-game/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Java-snake-game/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Java-snake-game/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Java-snake-game/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Java-snake-game/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Java-snake-game/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Java-snake-game/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Java-snake-game/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Java-snake-game/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Java-snake-game/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Java-snake-game/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Java-snake-game/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Java-snake-game/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Java-snake-game/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Java-snake-game/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Java-snake-game/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Java-snake-game/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Java-snake-game/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Java-snake-game/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Java-snake-game/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Java-snake-game/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Java-snake-game/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Java-snake-game/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Java-snake-game/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Java-snake-game/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Java-snake-game/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Java-snake-game/deployments',
    'created_at': '2022-12-22T18:58:38Z',
    'updated_at': '2023-01-16T10:35:02Z',
    'pushed_at': '2022-12-25T10:28:26Z',
    'git_url': 'git://github.com/shubanms/Java-snake-game.git',
    'ssh_url': 'git@github.com:shubanms/Java-snake-game.git',
    'clone_url': 'https://github.com/shubanms/Java-snake-game.git',
    'svn_url': 'https://github.com/shubanms/Java-snake-game',
    'homepage': None,
    'size': 5,
    'stargazers_count': 3,
    'watchers_count': 3,
    'language': 'Java',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 3,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 637491921,
    'node_id': 'R_kgDOJf9a0Q',
    'name': 'Machine-Learning-Lab',
    'full_name': 'shubanms/Machine-Learning-Lab',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Machine-Learning-Lab',
    'description': 'Practice for Machine Learning using the examples listed',
    'fork': True,
    'url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab',
    'forks_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Machine-Learning-Lab/deployments',
    'created_at': '2023-05-07T18:06:50Z',
    'updated_at': '2023-05-07T19:32:50Z',
    'pushed_at': '2023-05-07T19:36:12Z',
    'git_url': 'git://github.com/shubanms/Machine-Learning-Lab.git',
    'ssh_url': 'git@github.com:shubanms/Machine-Learning-Lab.git',
    'clone_url': 'https://github.com/shubanms/Machine-Learning-Lab.git',
    'svn_url': 'https://github.com/shubanms/Machine-Learning-Lab',
    'homepage': None,
    'size': 1614,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': 'Jupyter Notebook',
    'has_issues': False,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'master',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 598685557,
    'node_id': 'R_kgDOI683dQ',
    'name': 'markdown-cheatsheet',
    'full_name': 'shubanms/markdown-cheatsheet',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/markdown-cheatsheet',
    'description': 'A beginner markdown cheatsheet',
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet',
    'forks_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/markdown-cheatsheet/deployments',
    'created_at': '2023-02-07T15:59:51Z',
    'updated_at': '2023-02-07T17:18:30Z',
    'pushed_at': '2023-02-08T07:17:41Z',
    'git_url': 'git://github.com/shubanms/markdown-cheatsheet.git',
    'ssh_url': 'git@github.com:shubanms/markdown-cheatsheet.git',
    'clone_url': 'https://github.com/shubanms/markdown-cheatsheet.git',
    'svn_url': 'https://github.com/shubanms/markdown-cheatsheet',
    'homepage': None,
    'size': 41,
    'stargazers_count': 4,
    'watchers_count': 4,
    'language': None,
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'gpl-3.0',
      'name': 'GNU General Public License v3.0',
      'spdx_id': 'GPL-3.0',
      'url': 'https://api.github.com/licenses/gpl-3.0',
      'node_id': 'MDc6TGljZW5zZTk='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 4,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 747573815,
    'node_id': 'R_kgDOLI8SNw',
    'name': 'moodwave-backend',
    'full_name': 'shubanms/moodwave-backend',
    'private': True,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/moodwave-backend',
    'description': 'Official deployed moodwave backend',
    'fork': True,
    'url': 'https://api.github.com/repos/shubanms/moodwave-backend',
    'forks_url': 'https://api.github.com/repos/shubanms/moodwave-backend/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/moodwave-backend/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/moodwave-backend/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/moodwave-backend/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/moodwave-backend/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/moodwave-backend/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/moodwave-backend/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/moodwave-backend/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/moodwave-backend/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/moodwave-backend/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/moodwave-backend/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/moodwave-backend/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/moodwave-backend/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/moodwave-backend/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/moodwave-backend/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/moodwave-backend/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/moodwave-backend/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/moodwave-backend/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/moodwave-backend/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/moodwave-backend/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/moodwave-backend/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/moodwave-backend/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/moodwave-backend/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/moodwave-backend/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/moodwave-backend/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/moodwave-backend/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/moodwave-backend/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/moodwave-backend/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/moodwave-backend/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/moodwave-backend/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/moodwave-backend/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/moodwave-backend/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/moodwave-backend/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/moodwave-backend/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/moodwave-backend/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/moodwave-backend/deployments',
    'created_at': '2024-01-24T07:55:20Z',
    'updated_at': '2024-03-17T17:57:29Z',
    'pushed_at': '2023-12-25T05:42:29Z',
    'git_url': 'git://github.com/shubanms/moodwave-backend.git',
    'ssh_url': 'git@github.com:shubanms/moodwave-backend.git',
    'clone_url': 'https://github.com/shubanms/moodwave-backend.git',
    'svn_url': 'https://github.com/shubanms/moodwave-backend',
    'homepage': None,
    'size': 20,
    'stargazers_count': 1,
    'watchers_count': 1,
    'language': None,
    'has_issues': False,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': False,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': None,
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'private',
    'forks': 0,
    'open_issues': 0,
    'watchers': 1,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 773456875,
    'node_id': 'R_kgDOLhoD6w',
    'name': 'Rice-Classification',
    'full_name': 'shubanms/Rice-Classification',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/Rice-Classification',
    'description': None,
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/Rice-Classification',
    'forks_url': 'https://api.github.com/repos/shubanms/Rice-Classification/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/Rice-Classification/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/Rice-Classification/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/Rice-Classification/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/Rice-Classification/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/Rice-Classification/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/Rice-Classification/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/Rice-Classification/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/Rice-Classification/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/Rice-Classification/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/Rice-Classification/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/Rice-Classification/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/Rice-Classification/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/Rice-Classification/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/Rice-Classification/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/Rice-Classification/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/Rice-Classification/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/Rice-Classification/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/Rice-Classification/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/Rice-Classification/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/Rice-Classification/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/Rice-Classification/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/Rice-Classification/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/Rice-Classification/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/Rice-Classification/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/Rice-Classification/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/Rice-Classification/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/Rice-Classification/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/Rice-Classification/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/Rice-Classification/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/Rice-Classification/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/Rice-Classification/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/Rice-Classification/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/Rice-Classification/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/Rice-Classification/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/Rice-Classification/deployments',
    'created_at': '2024-03-17T17:58:50Z',
    'updated_at': '2024-03-17T18:00:44Z',
    'pushed_at': '2024-03-17T18:01:44Z',
    'git_url': 'git://github.com/shubanms/Rice-Classification.git',
    'ssh_url': 'git@github.com:shubanms/Rice-Classification.git',
    'clone_url': 'https://github.com/shubanms/Rice-Classification.git',
    'svn_url': 'https://github.com/shubanms/Rice-Classification',
    'homepage': None,
    'size': 22,
    'stargazers_count': 0,
    'watchers_count': 0,
    'language': 'Jupyter Notebook',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'gpl-3.0',
      'name': 'GNU General Public License v3.0',
      'spdx_id': 'GPL-3.0',
      'url': 'https://api.github.com/licenses/gpl-3.0',
      'node_id': 'MDc6TGljZW5zZTk='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 0,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  },
  {
    'id': 616650709,
    'node_id': 'R_kgDOJMFX1Q',
    'name': 'road_data_analysis',
    'full_name': 'shubanms/road_data_analysis',
    'private': False,
    'owner': {
      'login': 'shubanms',
      'id': 65527042,
      'node_id': 'MDQ6VXNlcjY1NTI3MDQy',
      'avatar_url': 'https://avatars.githubusercontent.com/u/65527042?v=4',
      'gravatar_id': '',
      'url': 'https://api.github.com/users/shubanms',
      'html_url': 'https://github.com/shubanms',
      'followers_url': 'https://api.github.com/users/shubanms/followers',
      'following_url': 'https://api.github.com/users/shubanms/following{/other_user}',
      'gists_url': 'https://api.github.com/users/shubanms/gists{/gist_id}',
      'starred_url': 'https://api.github.com/users/shubanms/starred{/owner}{/repo}',
      'subscriptions_url': 'https://api.github.com/users/shubanms/subscriptions',
      'organizations_url': 'https://api.github.com/users/shubanms/orgs',
      'repos_url': 'https://api.github.com/users/shubanms/repos',
      'events_url': 'https://api.github.com/users/shubanms/events{/privacy}',
      'received_events_url': 'https://api.github.com/users/shubanms/received_events',
      'type': 'User',
      'site_admin': False
    },
    'html_url': 'https://github.com/shubanms/road_data_analysis',
    'description': 'Analysis on Indian Roads and the density of states population and their co-relation with roads.',
    'fork': False,
    'url': 'https://api.github.com/repos/shubanms/road_data_analysis',
    'forks_url': 'https://api.github.com/repos/shubanms/road_data_analysis/forks',
    'keys_url': 'https://api.github.com/repos/shubanms/road_data_analysis/keys{/key_id}',
    'collaborators_url': 'https://api.github.com/repos/shubanms/road_data_analysis/collaborators{/collaborator}',
    'teams_url': 'https://api.github.com/repos/shubanms/road_data_analysis/teams',
    'hooks_url': 'https://api.github.com/repos/shubanms/road_data_analysis/hooks',
    'issue_events_url': 'https://api.github.com/repos/shubanms/road_data_analysis/issues/events{/number}',
    'events_url': 'https://api.github.com/repos/shubanms/road_data_analysis/events',
    'assignees_url': 'https://api.github.com/repos/shubanms/road_data_analysis/assignees{/user}',
    'branches_url': 'https://api.github.com/repos/shubanms/road_data_analysis/branches{/branch}',
    'tags_url': 'https://api.github.com/repos/shubanms/road_data_analysis/tags',
    'blobs_url': 'https://api.github.com/repos/shubanms/road_data_analysis/git/blobs{/sha}',
    'git_tags_url': 'https://api.github.com/repos/shubanms/road_data_analysis/git/tags{/sha}',
    'git_refs_url': 'https://api.github.com/repos/shubanms/road_data_analysis/git/refs{/sha}',
    'trees_url': 'https://api.github.com/repos/shubanms/road_data_analysis/git/trees{/sha}',
    'statuses_url': 'https://api.github.com/repos/shubanms/road_data_analysis/statuses/{sha}',
    'languages_url': 'https://api.github.com/repos/shubanms/road_data_analysis/languages',
    'stargazers_url': 'https://api.github.com/repos/shubanms/road_data_analysis/stargazers',
    'contributors_url': 'https://api.github.com/repos/shubanms/road_data_analysis/contributors',
    'subscribers_url': 'https://api.github.com/repos/shubanms/road_data_analysis/subscribers',
    'subscription_url': 'https://api.github.com/repos/shubanms/road_data_analysis/subscription',
    'commits_url': 'https://api.github.com/repos/shubanms/road_data_analysis/commits{/sha}',
    'git_commits_url': 'https://api.github.com/repos/shubanms/road_data_analysis/git/commits{/sha}',
    'comments_url': 'https://api.github.com/repos/shubanms/road_data_analysis/comments{/number}',
    'issue_comment_url': 'https://api.github.com/repos/shubanms/road_data_analysis/issues/comments{/number}',
    'contents_url': 'https://api.github.com/repos/shubanms/road_data_analysis/contents/{+path}',
    'compare_url': 'https://api.github.com/repos/shubanms/road_data_analysis/compare/{base}...{head}',
    'merges_url': 'https://api.github.com/repos/shubanms/road_data_analysis/merges',
    'archive_url': 'https://api.github.com/repos/shubanms/road_data_analysis/{archive_format}{/ref}',
    'downloads_url': 'https://api.github.com/repos/shubanms/road_data_analysis/downloads',
    'issues_url': 'https://api.github.com/repos/shubanms/road_data_analysis/issues{/number}',
    'pulls_url': 'https://api.github.com/repos/shubanms/road_data_analysis/pulls{/number}',
    'milestones_url': 'https://api.github.com/repos/shubanms/road_data_analysis/milestones{/number}',
    'notifications_url': 'https://api.github.com/repos/shubanms/road_data_analysis/notifications{?since,all,participating}',
    'labels_url': 'https://api.github.com/repos/shubanms/road_data_analysis/labels{/name}',
    'releases_url': 'https://api.github.com/repos/shubanms/road_data_analysis/releases{/id}',
    'deployments_url': 'https://api.github.com/repos/shubanms/road_data_analysis/deployments',
    'created_at': '2023-03-20T20:04:44Z',
    'updated_at': '2023-04-05T14:10:56Z',
    'pushed_at': '2023-03-21T07:34:24Z',
    'git_url': 'git://github.com/shubanms/road_data_analysis.git',
    'ssh_url': 'git@github.com:shubanms/road_data_analysis.git',
    'clone_url': 'https://github.com/shubanms/road_data_analysis.git',
    'svn_url': 'https://github.com/shubanms/road_data_analysis',
    'homepage': None,
    'size': 363,
    'stargazers_count': 3,
    'watchers_count': 3,
    'language': 'Jupyter Notebook',
    'has_issues': True,
    'has_projects': True,
    'has_downloads': True,
    'has_wiki': True,
    'has_pages': False,
    'has_discussions': False,
    'forks_count': 0,
    'mirror_url': None,
    'archived': False,
    'disabled': False,
    'open_issues_count': 0,
    'license': {
      'key': 'apache-2.0',
      'name': 'Apache License 2.0',
      'spdx_id': 'Apache-2.0',
      'url': 'https://api.github.com/licenses/apache-2.0',
      'node_id': 'MDc6TGljZW5zZTI='
    },
    'allow_forking': True,
    'is_template': False,
    'web_commit_signoff_required': False,
    'topics': [
      
    ],
    'visibility': 'public',
    'forks': 0,
    'open_issues': 0,
    'watchers': 3,
    'default_branch': 'main',
    'permissions': {
      'admin': True,
      'maintain': True,
      'push': True,
      'triage': True,
      'pull': True
    }
  }
]

In [3]:
for repo in now:
    print(repo['name'])

defy
GiAN
MoodWave-backend
SmartCalendar
mw-backend
algorithms
moodwave-backend
moodwave.github.io
mw-backend
blimp-total
CloudNine
Co2-emissions
Crop-Prediction
Deep-Learning
DL-Azure-Deployment
Dry-Beans
DSA
FERPlus
Food-Management-Systems
GDSC-DeathClock
gdsc-wow-hackathon
Geaze
GiAN-backend
HART
Java-snake-game
Machine-Learning-Lab
markdown-cheatsheet
moodwave-backend
Rice-Classification
road_data_analysis
